In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=23e22ac40d1b3377a4d2cb10f5590b778ec3662a8a63faebca3dac7ce7d7c23e
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [3]:
import os
from transformers import T5Tokenizer, TFT5ForConditionalGeneration
from rouge_score import rouge_scorer

In [4]:
# Load the T5 model and tokenizer
model_name = 't5-large'
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = TFT5ForConditionalGeneration.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


model.safetensors:   0%|          | 0.00/2.95G [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [5]:
# Function to read all text files in a folder with error handling
def read_text_files(folder_path):
    texts = []
    filenames = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".txt"):
            try:
                with open(os.path.join(folder_path, filename), 'r', encoding='utf-8', errors='ignore') as file:
                    texts.append(file.read())
                    filenames.append(filename)
            except Exception as e:
                print(f"Error reading file '{filename}': {e}")
    return texts, filenames

In [7]:
# Function to generate summaries
def generate_summary(text, max_length=150, min_length=40):
    inputs = tokenizer.encode("summarize: " + text, return_tensors='pt', max_length=512, truncation=True)
    summary_ids = model.generate(inputs, max_length=max_length, min_length=min_length, length_penalty=2.0, num_beams=4, early_stopping=True)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

In [8]:
# Function to evaluate ROUGE scores
def evaluate_summary(reference_summary, generated_summary):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(reference_summary, generated_summary)
    return scores

In [9]:
# Function to save generated summaries to files
def save_summaries(summaries, filenames, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    for summary, filename in zip(summaries, filenames):
        output_file = os.path.join(output_folder, f"{os.path.splitext(filename)[0]}_summary.txt")
        with open(output_file, 'w', encoding='utf-8') as file:
            file.write(summary)
        print(f"Generated summary saved: {output_file}")

In [10]:
# Path to the main folder containing subfolders for each category
main_folder_path = '/content/drive/MyDrive/Colab Notebooks/LLM Projects/BBC News Summary/News Articles'

# Path to the folder containing actual summaries
actual_summaries_folder = '/content/drive/MyDrive/Colab Notebooks/LLM Projects/BBC News Summary/Summaries'

# Output folder for saving generated summaries
output_folder = '/content/drive/MyDrive/Colab Notebooks/LLM Projects/BBC News Summary/t5-summaries'

In [12]:
# Process only the 'tech' category of bbc news articles
category = "tech"
category_path = os.path.join(main_folder_path, category)
if os.path.isdir(category_path):
    print(f"Processing category: {category}")
    articles, filenames = read_text_files(category_path)

    # Generate summaries for all articles in the category
    generated_summaries = [generate_summary(article) for article in articles]

    # Save generated summaries
    save_summaries(generated_summaries, filenames, os.path.join(output_folder, category))

    # Optionally, evaluate ROUGE scores
    actual_summaries, _ = read_text_files(os.path.join(actual_summaries_folder, category))
    total_scores = {'rouge1': [], 'rouge2': [], 'rougeL': []}
    for actual_summary, generated_summary in zip(actual_summaries, generated_summaries):
        scores = evaluate_summary(actual_summary, generated_summary)
        total_scores['rouge1'].append(scores['rouge1'].fmeasure)
        total_scores['rouge2'].append(scores['rouge2'].fmeasure)
        total_scores['rougeL'].append(scores['rougeL'].fmeasure)

    avg_scores = {
        'rouge1': sum(total_scores['rouge1']) / len(total_scores['rouge1']),
        'rouge2': sum(total_scores['rouge2']) / len(total_scores['rouge2']),
        'rougeL': sum(total_scores['rougeL']) / len(total_scores['rougeL'])
    }

    print(f"Average ROUGE-1: {avg_scores['rouge1']:.4f}")
    print(f"Average ROUGE-2: {avg_scores['rouge2']:.4f}")
    print(f"Average ROUGE-L: {avg_scores['rougeL']:.4f}")
    print("=" * 30)

print("Evaluation and saving completed.")

Processing category: tech
Generated summary saved: /content/drive/MyDrive/Colab Notebooks/LLM Projects/BBC News Summary/t5-summaries/tech/008_summary.txt
Generated summary saved: /content/drive/MyDrive/Colab Notebooks/LLM Projects/BBC News Summary/t5-summaries/tech/006_summary.txt
Generated summary saved: /content/drive/MyDrive/Colab Notebooks/LLM Projects/BBC News Summary/t5-summaries/tech/003_summary.txt
Generated summary saved: /content/drive/MyDrive/Colab Notebooks/LLM Projects/BBC News Summary/t5-summaries/tech/002_summary.txt
Generated summary saved: /content/drive/MyDrive/Colab Notebooks/LLM Projects/BBC News Summary/t5-summaries/tech/004_summary.txt
Generated summary saved: /content/drive/MyDrive/Colab Notebooks/LLM Projects/BBC News Summary/t5-summaries/tech/009_summary.txt
Generated summary saved: /content/drive/MyDrive/Colab Notebooks/LLM Projects/BBC News Summary/t5-summaries/tech/010_summary.txt
Generated summary saved: /content/drive/MyDrive/Colab Notebooks/LLM Projects/B